In [2]:
import cv2
import numpy as np
import glob
import random

In [3]:
net = cv2.dnn.readNet('yolov3_training_last.weights', 'yolov3_testing.cfg')

In [4]:
classes = ['pakhlava']

In [5]:
images_path = glob.glob('Pakhlava_test/raw/*.jpg')

In [6]:
save_path = glob.glob('Pakhlava_test/result/')

In [7]:
layer_names = net.getLayerNames()

In [8]:
netUnconOL = net.getUnconnectedOutLayers()

In [9]:
output_layers = [layer_names[i-1] for i in netUnconOL]

In [10]:
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [11]:
images_path, save_path

(['Pakhlava_test/raw\\0000.jpg',
  'Pakhlava_test/raw\\0001.jpg',
  'Pakhlava_test/raw\\0002.jpg',
  'Pakhlava_test/raw\\0003.jpg',
  'Pakhlava_test/raw\\0004-breaking.jpg'],
 ['Pakhlava_test/result/'])

In [12]:
random.shuffle(images_path)

In [21]:
for img_path in images_path:
    img = cv2.imread(img_path)
    img = cv2.resize(img, None, fx=0.4, fy=0.4)
    height, width, channels = img.shape
    
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    
    class_ids = []
    confidences = []
    boxes = []
    
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            if confidence > 0.3:
                print(class_id)
                center_x = int(detection[0]*width)
                center_y = int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)
                
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
                
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    print(indexes)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = str(confidences[i])[:4]
            #color = colors[class_ids[i]]
            color = np.random.uniform(0, 255, size=(len(classes), 3))[0]
            cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
            cv2.putText(img, label+': '+confidence, (x, y + 33), font, 1, color, 2)
    
    cv2.imwrite(save_path[0]+img_path.split('/')[1].split('\\')[1], img)
    cv2.imshow('Image', img)
    cv2.waitKey(0)
cv2.destroyAllWindows()

0
0
0
0
[0 1 3 2]
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
[ 9  5 20 11 22  2  7 13 15 18 19 16  1  0  4]
0
0
[1]
0
0
0
0
[1 3]
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
[ 3 15  8  6 13  4 10]
